In [ ]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet import ResNet50
from keras.applications.resnet import ResNet101
from keras.applications.resnet import ResNet152
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.resnet_v2 import ResNet101V2
from keras.applications.resnet_v2 import ResNet152V2
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import DenseNet169
from keras.applications.densenet import DenseNet201
from keras.activations import softmax, relu, sigmoid
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint,TensorBoard,ReduceLROnPlateau,CSVLogger, EarlyStopping
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import cv2
from random import shuffle
import random
import glob
from skimage.transform import resize
#from PIL import Image
import os
#from model import *

from collections import defaultdict
import itertools
from tqdm import tqdm
import time
import shutil



In [ ]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
set_session(sess)

In [ ]:
model_name = 'VGG16_Jan2020'
num_classes = 14

In [ ]:
def batch_generator(data_X, data_y, batch_size):
    indexes = np.array(range(len(data_y)))
    n = len(indexes)
    while True:
        batch_start = 0
        batch_end = batch_size
        np.random.shuffle(indexes)
        while batch_start < n:
            index = []
            batch_y = []
            y = []
            index = indexes[batch_start:batch_end]
            batch_x = np.array([data_X[i] for i in index])
            batch_y = np.array([data_y[i] for i in index])
            yield batch_x, batch_y
            batch_start += batch_size   
            batch_end += batch_size
            if (batch_end>len(data_y)):
                batch_end = len(data_y)

In [ ]:
def read_landUse(dataset_path, im_type):
    import glob
    paths_X = []   
    labels = [] 
    i = 0
    folders = sorted(os.listdir(dataset_path))
    for folder in folders:
        temp = sorted (glob.glob(os.path.join(dataset_path,folder+'/*'+im_type)))
        for k in range(len(temp)):
            labels.append (i)
        paths_X += temp
        i += 1
        
    if(len(paths_X)==0):
        print ('Dataset could not found. Please provide correct path.')
    return paths_X, labels

In [ ]:
dataset_path = 'Dataset/train/'
paths, labels = read_landUse(dataset_path, '[(.png)(.jpg)]')
images = np.array([cv2.imread(image_name) for image_name in paths]) 

In [ ]:
X_train = images
y_train = to_categorical(labels, num_classes)
#X, y = load_dataset(dataset_path)
print ('Size of dataset:', len(y_train))

In [ ]:
dataset_path_val = 'Dataset/val/'
paths_val, labels_val = read_landUse(dataset_path_val, '[(.png)(.jpg)]')
X_val =  np.array([cv2.imread(image_name) for image_name in paths_val]) 
y_val = to_categorical(labels_val, num_classes)

In [ ]:
#X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.2)

Display dataset stats

In [ ]:
folders = os.listdir(dataset_path)
names, counts = [], []
for i, folder in enumerate(folders):
    dir_path = os.path.join(dataset_path, folder)
    images = os.listdir(dir_path)
    names.append(folder)
    counts.append(len(images))
    print(i, folder, len(images))
df = pd.DataFrame(data={'Name': names, 'Count': counts})
ax = df.plot(kind='bar', xticks=counts, grid=True, legend=False, figsize=(16,6))
ax.set_xlabel('Classes', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
plt.xticks(np.arange(len(names)), names, rotation=45, fontsize=15)
for i, v in enumerate(counts):
    ax.text(i-.18, v+15, str(v), color='blue', fontweight='bold')
plt.show()

In [ ]:
batch_size = 10
train_gen = batch_generator(X_train, y_train, batch_size)
val_gen = batch_generator(X_val, y_val, batch_size)
#test_gen = batch_generator(X_test, y_test, batch_size)

In [ ]:
input_shape=(256,256,3)
model= VGG16(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
#model= VGG19(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
#model= ResNet50(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
#= ResNet101(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
#model= ResNet152(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
#model= ResNet50V2(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
#model= ResNet101V2(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
#model= ResNet152V2(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
#model= InceptionResNetV2(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
#model= InceptionV3(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
#model= DenseNet121(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
#model= DenseNet169(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
#model= DenseNet201(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
#model =InceptionResNetV2(include_top=True, weights=None, input_tensor=None, input_shape=input_shape, pooling=None, classes=num_classes)
#model = resNet(input_shape, num_classes, model_type='resnet_152')
#model.load_weights('WEIGHTS/'+model_name+'.h5')

### model_vgg16.layers[-1].activation = relu

### idx_of_layer_to_change = -1
#model.layers[idx_of_layer_to_change].activation = activations.softmax
#model = utils.apply_modifications(model)

#model_vgg16.layers[-1].activation = relu
# # FINE TUNING HERE
# top_model = Sequential()
# top_model.add(Dense(input_shape=model.layers[-2].output_shape, units=num_classes, rnel
# kernel_initializer="he_normal", activation="softmax"))

# model.layers.pop()
# model.outputs = [model.layers[-1].output]
# model.layers[-1].outbound_nodes = []

# model = Model(inputs=model.inputs, outputs=top_model(model.outputs[0]))
# # for layer in model.layers[:-1]:
# #     layer.trainable = False
#model.load_weights('WEIGHTS/'+model_name+'.h5')

model.summary()
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
# plot_model(model, to_file='model.png')


In [ ]:
class_weights = {0: 629/650, 
                 1: 629/635, 
                 2: 629/634, 
                 3: 629/630, 
                 4: 629/629, 
                 5: 629/633, 
                 6: 629/629, 
                 7: 629/620, 
                 8: 629/632, 
                 9: 629/617, 
                 10: 629/623,
                 11:629/638,
                 12:629/635,
                 13:629/524
                }

Train ResNet-152 network using "fit_generator"

In [ ]:
checkpoint = ModelCheckpoint('WEIGHTS/'+model_name+'.h5', monitor='val_loss', verbose=2, save_best_only=True, mode='auto')
tensorboard = TensorBoard(log_dir='EVENTS/', batch_size=batch_size, write_graph=True, write_images=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3, min_lr=0.00001)
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=2, mode='auto')
cvslogger = keras.callbacks.CSVLogger('WEIGHTS/'+model_name+'.csv', separator=',', append=True)
 
nb_epoch = 200
callbacks = [checkpoint, reduce_lr, earlystop, cvslogger, tensorboard]
train_steps = int(len(y_train)//batch_size)
val_steps = int(len(y_val)//batch_size)
start_time = time.time()
history = model.fit_generator(train_gen, train_steps, epochs=nb_epoch, verbose=1, 
                    max_queue_size=2, validation_data=val_gen, 
                    validation_steps=val_steps, shuffle=True,
                    workers=1, use_multiprocessing= False, 
                    initial_epoch=0, callbacks=callbacks, class_weight=class_weights)
print("Seconds: ", time.time() - start_time)
#model.fit(x=X[0:100], y=y[0:100], batch_size=64, epochs=2, verbose=1, callbacks=callbacks, 
#          validation_split=0.2, shuffle=True,initial_epoch=0)
